In [34]:
REFERENCE_SIZES = {
    'BusinessDepartment': {'height': 15.0, 'width': 40.0, 'reference_pixel_height': 450},
    'Library': {'height': 11.0, 'width': 45.0, 'reference_pixel_height': 450},
    'EEDepartment': {'height': 12.0, 'width': 50.0, 'reference_pixel_height': 450},
    'CivilDepartment': {'height': 14.0, 'width': 50.0, 'reference_pixel_height': 500},
    'OldCSDepartment': {'height': 15.0, 'width': 48.0, 'reference_pixel_height': 450},
    'NewCSDepartment': {'height': 16.0, 'width': 50.0, 'reference_pixel_height': 500}
}

In [35]:
def load_recognition_model():
    """Loads the trained landmark recognition model"""
    import os
    import pandas as pd
    from tensorflow.keras.models import load_model
    import tensorflow as tf
    from sklearn.preprocessing import LabelEncoder
    # from google.colab import files

    model_path = 'model/landmark_recognition_model.h5'
    if not os.path.exists(model_path):
        model_path = 'landmark_recognition_model.h5'
        if not os.path.exists(model_path):
            print("Model file not found. Please upload your model file:")
            uploaded = files.upload()
            model_path = list(uploaded.keys())[0]

    model = load_model(model_path)

    annotations_path = "annotations/annotations.csv"
    if not os.path.exists(annotations_path):
        annotations_path = "annotations.csv"
        if not os.path.exists(annotations_path):
            print("Annotations CSV not found. Please upload your annotations file:")
            uploaded = files.upload()
            annotations_path = list(uploaded.keys())[0]

    df = pd.read_csv(annotations_path)
    label_encoder = LabelEncoder()
    label_encoder.fit(df['building_name'])

    base_model = tf.keras.applications.MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=(224, 224, 3),
        pooling='avg'
    )
    base_model.trainable = False

    return model, label_encoder, base_model

In [36]:
def load_and_preprocess_image(img_path):
    """Load and preprocess image for the model"""
    from tensorflow.keras.preprocessing import image
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
    import numpy as np

    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    return img_array, img

In [37]:
def recognize_landmark(img_path, model, label_encoder, base_model):
    """Recognize landmark in the given image"""
    import numpy as np

    preprocessed_img, original_img = load_and_preprocess_image(img_path)
    features = base_model.predict(preprocessed_img)
    prediction = model.predict(features)
    predicted_class = np.argmax(prediction[0])
    building_name = label_encoder.classes_[predicted_class]
    confidence = prediction[0][predicted_class]

    return building_name, confidence, original_img

In [38]:
def detect_building_boundaries(image_path):
    """Detect the boundaries of a building in the image"""
    import cv2
    import numpy as np

    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image at path: {image_path}")
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    dilated = cv2.dilate(edges, np.ones((3, 3), np.uint8), iterations=2)
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return None, 0, img_rgb

    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    cv2.rectangle(img_rgb, (x, y), (x+w, y+h), (0, 255, 0), 2)

    return (x, y, w, h), h, img_rgb

In [39]:
def estimate_distance_by_size(building_name, pixel_height):
    """Estimate the distance to a building based on its apparent size in pixels"""
    if building_name not in REFERENCE_SIZES:
        return None

    reference_data = REFERENCE_SIZES[building_name]
    reference_height_m = reference_data['height']
    reference_pixel_height = reference_data['reference_pixel_height']
    reference_distance = 10.0  # meters
    focal_length = (reference_pixel_height * reference_distance) / reference_height_m
    estimated_distance = (reference_height_m * focal_length) / pixel_height

    return estimated_distance

In [40]:
def estimate_location_by_trilateration(buildings, distances):
    """Estimate user location using trilateration from three buildings"""
    import numpy as np

    BUILDING_COORDINATES = {
        'BusinessDepartment': (100, 200),
        'EEDepartment': (280, 400),
        'Library': (300, 150),
        'OldCSDepartment': (250, 350),
        'CivilDepartment': (150, 300),
        'NewCSDepartment': (400, 250)
    }

    if len(buildings) != 3 or len(distances) != 3:
        return None

    for building in buildings:
        if building not in BUILDING_COORDINATES:
            return None

    # Extract coordinates and distances
    x1, y1 = BUILDING_COORDINATES[buildings[0]]
    x2, y2 = BUILDING_COORDINATES[buildings[1]]
    x3, y3 = BUILDING_COORDINATES[buildings[2]]
    r1, r2, r3 = distances

    # Trilateration equations: solve for (x, y)
    # (x - x1)^2 + (y - y1)^2 = r1^2
    # (x - x2)^2 + (y - y2)^2 = r2^2
    # (x - x3)^2 + (y - y3)^2 = r3^2
    # Use linearization to solve
    A = np.array([
        [2*(x1 - x3), 2*(y1 - y3)],
        [2*(x2 - x3), 2*(y2 - y3)]
    ])
    B = np.array([
        [r3**2 - r1**2 - x3**2 + x1**2 - y3**2 + y1**2],
        [r3**2 - r2**2 - x3**2 + x2**2 - y3**2 + y2**2]
    ])

    try:
        X = np.linalg.solve(A, B)
        x, y = X[0][0], X[1][0]
        return (x, y)
    except np.linalg.LinAlgError:
        return None

In [41]:
def visualize_results(img_rgb, building_name, confidence, distance):
    """Display the image with detected building and estimated distance"""
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 8))
    plt.imshow(img_rgb)
    plt.title(f"Detected: {building_name} (Confidence: {confidence:.2f})")
    plt.xlabel(f"Estimated Distance: {distance:.2f} meters")
    plt.axis('off')
    plt.show()

In [42]:
def estimate_location(building_names, distances, bearings=None):
    """Estimate user's location based on distances to landmarks"""
    import math

    BUILDING_COORDINATES = {
        'BusinessDepartment': (100, 200),
        'EEDepartment': (280, 400),
        'Library': (300, 150),
        'OldCSDepartment': (250, 350),
        'CivilDepartment': (150, 300),
        'NewCSDepartment': (400, 250)
    }

    if len(building_names) >= 3:
        # Use trilateration for three or more buildings
        return estimate_location_by_trilateration(building_names[:3], distances[:3])
    elif len(building_names) == 1:
        # Fallback to single building estimation
        building_name = building_names[0]
        distance = distances[0]
        if building_name not in BUILDING_COORDINATES:
            return None

        building_x, building_y = BUILDING_COORDINATES[building_name]
        if bearings is None:
            angle_rad = math.radians(180)  # Default to south
        else:
            angle_rad = math.radians(bearings[0])

        x = building_x - distance * math.sin(angle_rad)
        y = building_y + distance * math.cos(angle_rad)
        return x, y
    else:
        return None

In [43]:
def visualize_location_on_map(user_location, building_coordinates, detected_building=None, filename=None):
    import folium
    import os
    import uuid

    m = folium.Map(location=user_location, zoom_start=18)
    folium.Marker(user_location, tooltip="Estimated Location", icon=folium.Icon(color="red")).add_to(m)

    for building_name, coords in building_coordinates.items():
        folium.Marker(
            location=coords,
            tooltip=building_name,
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(m)

    if detected_building and detected_building in building_coordinates:
        folium.PolyLine(locations=[user_location, building_coordinates[detected_building]],
                        color='green', weight=2.5, opacity=0.8).add_to(m)

    if not filename:
        filename = f"map_{uuid.uuid4().hex}.html"
    save_path = os.path.join("static", "maps", filename)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    m.save(save_path)

    return filename

In [ ]:
from flask import Flask, request, jsonify
import nest_asyncio
from pyngrok import ngrok
import os
import uuid

app = Flask(__name__)

@app.route('/process-images', methods=['POST'])
def process_images():
    model, label_encoder, base_model = load_recognition_model()
    BUILDING_COORDINATES = {
        'BusinessDepartment': (100, 200),
        'EEDepartment': (280, 400),
        'Library': (300, 150),
        'OldCSDepartment': (250, 350),
        'CivilDepartment': (150, 300),
        'NewCSDepartment': (400, 250)
    }

    if 'images' not in request.files:
        return jsonify({'error': 'No images provided'}), 400

    images = request.files.getlist('images')
    if len(images) < 1 or len(images) > 3:
        return jsonify({'error': 'Please provide 1 to 3 images'}), 400

    results = []
    building_names = []
    distances = []

    for idx, img_file in enumerate(images):
        img_path = f'temp_image_{uuid.uuid4().hex}.jpg'
        img_file.save(img_path)

        try:
            building_name, confidence, original_img = recognize_landmark(img_path, model, label_encoder, base_model)
            bbox, pixel_height, img_rgb = detect_building_boundaries(img_path)
            distance = estimate_distance_by_size(building_name, pixel_height) if pixel_height > 0 else None

            result = {
                'image_index': idx,
                'building_name': building_name,
                'confidence': str(confidence),
                'bounding_box': str(bbox),
                'pixel_height': str(pixel_height),
                'distance': str(distance) if distance else 'None'
            }

            building_names.append(building_name)
            distances.append(distance if distance else 0)
            results.append(result)

        finally:
            if os.path.exists(img_path):
                os.remove(img_path)

    user_location = estimate_location(building_names, distances)
    print(user_location)
    map_filename = None
    if user_location:
        map_filename = visualize_location_on_map(user_location, BUILDING_COORDINATES, building_names[0] if len(building_names) == 1 else None)
        map_url = f"http://127.0.0.1:5000/static/maps/{map_filename}"
    else:
        map_url = 'None'

    response = {
        'results': results,
        'user_location': str(user_location) if user_location else 'None',
        'map_url': map_url
    }
    print(response)
    return jsonify(response)

if __name__ == '__main__':
    nest_asyncio.apply()
    app.run(host='0.0.0.0', port=5000, debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.18:5000
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.18:5000
Press CTRL+C to quit
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


192.168.1.17 - - [13/May/2025 23:57:58] "POST /process-images HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.17 - - [13/May/2025 23:57:58] "POST /process-images HTTP/1.1" 200 -


None
{'results': [{'image_index': 0, 'building_name': 'EEDepartment', 'confidence': '0.8130387', 'bounding_box': '(1712, 2182, 870, 157)', 'pixel_height': '157', 'distance': '28.662420382165607'}, {'image_index': 1, 'building_name': 'BusinessDepartment', 'confidence': '0.638093', 'bounding_box': '(2868, 411, 209, 1358)', 'pixel_height': '1358', 'distance': '3.313696612665685'}, {'image_index': 2, 'building_name': 'BusinessDepartment', 'confidence': '0.86560667', 'bounding_box': '(324, 2526, 3132, 56)', 'pixel_height': '56', 'distance': '80.35714285714286'}], 'user_location': 'None', 'map_url': 'None'}
